In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.cluster.hierarchy as sch
import util.beam_data as bd
from util.data import find_last_created_dir
from util.data import meaningful

schema = {
    "type": str,
    "sender_parent": str,
    "sender_name": str,
    "receiver_parent": str,
    "receiver_name": str,
    "payload": str,
    "state": str,
    "tick": int,
    "triggerId": pd.Int64Dtype(),
}
root = "../beam_root"
# beam.debug.messageLogging = true (in beam.conf)

msg_file_num = 0
def load_data(dir, it):
    return pd.read_csv(f"{dir}/ITERS/it.{it}/{it}.actor_messages_{msg_file_num}.csv.gz", dtype=schema)
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
# beam_out_path = "beamville/rh-reservation-before-walking__2023-02-09_18-36-57_bjq" #good
beam_out_path = "beamville/rh-stop-early-reservation-default-manager__2023-02-14_19-50-15_kzc"   #bad
beam_out_path = None
beam_out = find_last_created_dir(f"{root}/output", level = 1, num = 0) if beam_out_path is None else f"{root}/output/{beam_out_path}"
print(f"Using beam out dir: {beam_out}")

ifields = ['type', 'sender_name', 'receiver_name', 'payload', 'state', 'tick', 'triggerId']

msg = load_data(beam_out, 0)
# msg = load_data('enroute-ci-stuck__2022-02-01_11-07-10_pxv', 0)
msg.head()

Using beam out dir: ../output/sf-light/rhtp-develop-with-rh-stops__2023-10-16_12-48-56_fie


,type,sender_parent,sender_name,receiver_parent,receiver_name,payload,state,tick,triggerId
0,message,NaN,BeamMobsim.iteration,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
1,message,NaN,BeamMobsim.iteration,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
2,message,NaN,BeamMobsim.iteration,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
3,message,NaN,BeamMobsim.iteration,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
4,message,NaN,BeamMobsim.iteration,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1


In [2]:
person_id = '012202-2013000609715-0-5899892'
display(f"-- Person sequence: {person_id} --")

trigger_ids = msg[((msg['sender_name'] == person_id) & (msg['sender_parent'] != "population")) | ((msg['receiver_name'] == person_id) & (msg['receiver_parent'] != "population"))
                  | (msg['payload'].str.contains(f"/{person_id}#"))
]['triggerId'].unique()
trigger_ids = [id for id in trigger_ids if id >= 0]

person4 = msg[
    (
            ((msg['sender_name'] == person_id) & (msg['sender_parent'] != "population"))
            | ((msg['receiver_name'] == person_id) & (msg['receiver_parent'] != "population"))
            | (msg['triggerId'].isin(trigger_ids))
            | (msg['payload'].str.contains(f"/{person_id}#"))
    )
    # & (msg['payload'].str.startswith('Routing'))
]
# ].shape
# ].iloc[5]['payload']
# person4[person4['payload'].str.contains('Board')]
person4

'-- Person sequence: 012202-2013000609715-0-5899892 --'

,type,sender_parent,sender_name,receiver_parent,receiver_name,payload,state,tick,triggerId
7474,message,population,012202-2013000609715-0,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
7644,event,BeamMobsim.iteration,scheduler,population/012202-2013000609715-0,012202-2013000609715-0-5899892,"TriggerWithId(InitializeTrigger(0),2592)","BasePersonData(0,None,List(),Vector(),None,Non...",0,2592
7646,transition,BeamMobsim.iteration,scheduler,population/012202-2013000609715-0,012202-2013000609715-0-5899892,Uninitialized,Initialized,-1,-94677838
18245,message,population/012202-2013000609715-0,012202-2013000609715-0-5899892,BeamMobsim.iteration,scheduler,"CompletionNotice(2592,Vector(ScheduleTrigger(A...",NaN,-1,2592
18276,event,BeamMobsim.iteration,scheduler,population/012202-2013000609715-0,012202-2013000609715-0-5899892,"TriggerWithId(ActivityStartTrigger(0),5604)","BasePersonData(0,None,List(),Vector(),None,Non...",0,5604
...,...,...,...,...,...,...,...,...,...
2599706,message,population/012202-2013000609715-0,012202-2013000609715-0-5899892,BeamMobsim.iteration,scheduler,"CompletionNotice(656649,Vector(ScheduleTrigger...",NaN,-1,656649
2634230,event,BeamMobsim.iteration,scheduler,population/012202-2013000609715-0,012202-2013000609715-0-5899892,"TriggerWithId(ActivityEndTrigger(67668),727549)","BasePersonData(2,None,List(),Vector(),None,Non...",67668,727549
2634276,message,population/012202-2013000609715-0,012202-2013000609715-0-5899892,BeamMobsim.iteration,scheduler,"CompletionNotice(727549,Vector())",NaN,-1,727549
3366986,event,population,012202-2013000609715-0,population/012202-2013000609715-0,012202-2013000609715-0-5899892,Finish,"BasePersonData(2,None,List(),Vector(),None,Non...",-1,-1


In [47]:
res1 = msg[msg['payload'].str.contains('RideHailResponse.+error: None', case=False)]
res1
# msg.tail()

,type,sender_parent,sender_name,receiver_parent,receiver_name,payload,state,tick,triggerId
105400,message,RideHailManager,RideHailManager-Uber,BeamMobsim.iteration,RideHailManager,RideHailResponse(request: RideHailRequest(id: ...,NaN,-1,2356755
105402,event,BeamMobsim.iteration,RideHailManager,population/525259,1377748,RideHailResponse(request: RideHailRequest(id: ...,"ChoosesModeData(BasePersonData(0,None,List(),V...",24789,2356755
105675,message,RideHailManager,RideHailManager-Lyft,BeamMobsim.iteration,RideHailManager,RideHailResponse(request: RideHailRequest(id: ...,NaN,-1,5897
106544,message,RideHailManager,RideHailManager-Lyft,BeamMobsim.iteration,RideHailManager,RideHailResponse(request: RideHailRequest(id: ...,NaN,-1,2130627
107184,message,RideHailManager,RideHailManager-Uber,BeamMobsim.iteration,RideHailManager,RideHailResponse(request: RideHailRequest(id: ...,NaN,-1,2671420
...,...,...,...,...,...,...,...,...,...
9999911,event,BeamMobsim.iteration,RideHailManager,population/464695,1209196,RideHailResponse(request: RideHailRequest(id: ...,"ChoosesModeData(BasePersonData(0,None,List(),V...",27082,2450513
9999947,event,BeamMobsim.iteration,RideHailManager,population/1535084,3889337,RideHailResponse(request: RideHailRequest(id: ...,"ChoosesModeData(BasePersonData(0,None,List(),V...",27039,2800264
9999969,event,BeamMobsim.iteration,RideHailManager,population/196488,497292,RideHailResponse(request: RideHailRequest(id: ...,"ChoosesModeData(BasePersonData(0,None,List(),V...",27050,2745999
9999989,event,BeamMobsim.iteration,RideHailManager,population/97883,241487,RideHailResponse(request: RideHailRequest(id: ...,"ChoosesModeData(BasePersonData(0,None,List(),V...",27082,1560843


In [48]:
res1.iloc[0, :]['payload']

'RideHailResponse(request: RideHailRequest(id: 118849, type: RideHailInquiry, customer: 1377748, pickup: [x=607932.555802223][y=4172950.4093762757], time: 26100, dest: [x=608715.9725562811][y=4171481.7495681765]), error: None, travelProposal: Some(RHA: rideHailVehicle-1404355@Uber, price: Map(1377748 -> 10.141413880256888), passengerSchedule: BeamLeg(CAR @ 26100,dur:198,path: 95573 .. 74625) -> [0riders;0boarders;0alighters]--BeamLeg(CAR @ 26298,dur:82,path: 74624 .. 81151) -> [1riders;1boarders;1alighters]))'

In [4]:
# rh_agent = msg[msg['payload'].str.contains("StartLegTrigger(22500", regex=False)]
# display(rh_agent)
# rh_agent
msg[msg['triggerId'] == 83015]

,type,sender_parent,sender_name,receiver_parent,receiver_name,payload,state,tick,triggerId
256799,event,BeamMobsim.iteration,scheduler,transit-system,TransitDriverAgent-BA:10SFO10,"TriggerWithId(StartLegTrigger(24480,BeamLeg(SU...","TransitDriverData(null,Vector(BA:10SFO10),Beam...",24480,83015
257010,message,transit-system,TransitDriverAgent-BA:10SFO10,BeamMobsim.iteration,scheduler,"CompletionNotice(83015,Vector(ScheduleTrigger(...",NaN,-1,83015


In [4]:
# RideHailManager interaction
rh = msg[(msg['sender_name'].str.contains("RideHailManager")) | (msg['receiver_name'].str.contains("RideHailManager"))]
# rh[rh['tick'] == 22500]
rh.loc[7200:9000]

,type,sender_parent,sender_name,receiver_parent,receiver_name,payload,state,tick,triggerId
6191,message,BeamMobsim.iteration,RideHailManager,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
6192,message,BeamMobsim.iteration,RideHailManager,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
6193,message,BeamMobsim.iteration,RideHailManager,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
6194,message,BeamMobsim.iteration,RideHailManager,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
6195,message,BeamMobsim.iteration,RideHailManager,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
...,...,...,...,...,...,...,...,...,...
64311,event,BeamMobsim.iteration,RideHailManager,RideHailManager,rideHailAgent-accessible-31,Finish,RideHailAgentData(rideHailVehicle-31@GlobalRHM...,-1,-1
64312,message,NaN,BeamMobsim.iteration,BeamMobsim.iteration,RideHailManager,Finish,NaN,-1,-1
64313,message,RideHailManager,rideHailAgent-accessible-24,BeamMobsim.iteration,RideHailManager,Terminated(Actor[akka://ClusterSystem/user/Bea...,NaN,-1,-1
64314,message,RideHailManager,rideHailAgent-accessible-43,BeamMobsim.iteration,RideHailManager,Terminated(Actor[akka://ClusterSystem/user/Bea...,NaN,-1,-1


In [8]:
# msg[(msg['triggerId'].isin([2999])) | msg['payload'].str.contains('-228-L5', na=False) | msg['state'].str.contains('-228-L5', na=False)][ifields]
msg.iloc[63900:64000]

,type,sender_parent,sender_name,receiver_parent,receiver_name,payload,state,tick,triggerId
63900,message,RideHailManager,RideHailManager-GlobalRHM,BeamMobsim.iteration,scheduler,"CompletionNotice(15143,Vector(ScheduleTrigger(...",NaN,-1,15143
63901,message,BeamMobsim.iteration,scheduler,RideHailManager,RideHailManager-GlobalRHM,TriggerWithId(RideHailRepositioningTrigger(107...,NaN,107250,15142
63902,message,RideHailManager,RideHailManager-GlobalRHM,RideHailManager,RideHailManager-GlobalRHM,"ParkingStallsClaimedByVehicles(107250,Vector()...",NaN,-1,15142
63903,message,RideHailManager,RideHailManager-GlobalRHM,BeamMobsim.iteration,scheduler,"CompletionNotice(15142,Vector(ScheduleTrigger(...",NaN,-1,15142
63904,message,BeamMobsim.iteration,scheduler,BeamMobsim.iteration,ChargingNetworkManager,TriggerWithId(PlanEnergyDispatchTrigger(107400...,NaN,107400,15145
...,...,...,...,...,...,...,...,...,...
63995,message,RideHailManager/RideHailManager-GlobalRHM,rideHailAgent-2,RideHailManager,RideHailManager-GlobalRHM,Terminated(Actor[akka://ClusterSystem/user/Bea...,NaN,-1,-1
63996,event,population,12,population/12,28,Finish,"BasePersonData(2,None,List(),Vector(),None,Non...",-1,-1
63997,message,population/19,44,population,19,Terminated(Actor[akka://ClusterSystem/user/Bea...,NaN,-1,-1
63998,message,population/19,45,population,19,Terminated(Actor[akka://ClusterSystem/user/Bea...,NaN,-1,-1


In [21]:
msg.loc[1942245]['payload']

'ChargingPlugRequest(29254,virtual-personalVehicle-27512 (ev-L1-0-to-50000-LowTech-2050-Compact-BEV_200_XFC-400,257.28022877325816km),ParkingStall(591,4726,[x=602177.2508767622][y=4133585.352729465],0.16366000000000003,Some(publicxfc(300.0|DC)),Some(Block),Public,default(Any)),virtual-personalVehicle-27512,584124,Actor[akka://ClusterSystem/user/BeamMobsim.iteration/ChargingNetworkManager#1010989642],NotApplicable,None)'

In [21]:
msg[msg['payload'].str.contains('virtual-personalVehicle-2905')].iloc[1]['payload']

'TriggerWithId(PlanParkingInquiryTrigger(23808,ParkingInquiry(SpaceTime([x=585419.6202943256][y=4234654.94731052],23808),EnRoute-school,default(Any),Some(virtual-personalVehicle-2905 (ev-L1-0-to-50000-LowTech-2050-Pickup-BEV_200_XFC-400,257.6477096040279km)),None,Some(virtual-personalVehicle-2905),1.0,1096.0,true,6313,DestinationCharging,None,227592)),239043)'

In [7]:
# person4['receiver_parent'].unique()
# msg.loc[8848726]['payload']
msg[msg['payload'].str.startswith("CompletionNotice(1080911")]

,type,sender_parent,sender_name,receiver_parent,receiver_name,payload,state,tick,triggerId
4663164,message,population/1253024,971174,BeamMobsim.iteration,scheduler,"CompletionNotice(1080911,Vector(ScheduleTrigge...",NaN,-1,1080911
4663540,message,population/1253024,971174,BeamMobsim.iteration,scheduler,"CompletionNotice(1080911,Vector(ScheduleTrigge...",NaN,-1,1080911


In [6]:
agent_id = 'rideHailAgent-047701-2013000697331-0-433937'
# agent_id = 'TransitDriverAgent-SF:7590302'
display(f"-- Ride Hail Agent sequence {agent_id} --")

trigger_ids = msg[(msg['sender_name'] == agent_id) | (msg['receiver_name'] == agent_id)]['triggerId'].unique()
trigger_ids = [id for id in trigger_ids if id >= 0]

allAgents = msg[
    (
            (msg['sender_name'] == agent_id)
            | (msg['receiver_name'] == agent_id)
            | (msg['triggerId'].isin(trigger_ids))
    )
]
allAgents[(
    ~((allAgents['sender_name'].str.startswith('rideHailAgent-') & (allAgents['sender_name'] != agent_id))
      | (allAgents['receiver_name'].str.startswith('rideHailAgent-') & (allAgents['receiver_name'] != agent_id))
      )
)
]
#.iloc[0:60]
    # ].shape
# ].to_csv("../docs/uml/ride-hail-agent.csv")
# ].iloc[5]['payload']

'-- Ride Hail Agent sequence rideHailAgent-047701-2013000697331-0-433937 --'

,type,sender_parent,sender_name,receiver_parent,receiver_name,payload,state,tick,triggerId
24904,event,BeamMobsim.iteration,scheduler,RideHailManager,rideHailAgent-047701-2013000697331-0-433937,"TriggerWithId(InitializeTrigger(0),8771)",RideHailAgentData(rideHailVehicle-047701-20130...,0,8771
24905,message,RideHailManager,rideHailAgent-047701-2013000697331-0-433937,BeamMobsim.iteration,scheduler,"CompletionNotice(8771,Vector(ScheduleTrigger(S...",NaN,-1,8771
24906,transition,BeamMobsim.iteration,scheduler,RideHailManager,rideHailAgent-047701-2013000697331-0-433937,Uninitialized,Offline,-1,-1588475857
1094491,event,BeamMobsim.iteration,scheduler,RideHailManager,rideHailAgent-047701-2013000697331-0-433937,"TriggerWithId(StartShiftTrigger(38758),8774)",RideHailAgentData(rideHailVehicle-047701-20130...,38758,8774
1094514,transition,BeamMobsim.iteration,scheduler,RideHailManager,rideHailAgent-047701-2013000697331-0-433937,Offline,Idle,38758,8774
...,...,...,...,...,...,...,...,...,...
2541748,event,BeamMobsim.iteration,scheduler,RideHailManager,rideHailAgent-047701-2013000697331-0-433937,"TriggerWithId(EndShiftTrigger(65882),310374)",RideHailAgentData(rideHailVehicle-047701-20130...,65882,310374
2541749,transition,BeamMobsim.iteration,scheduler,RideHailManager,rideHailAgent-047701-2013000697331-0-433937,Idle,Offline,-1,-411065503
2541751,message,RideHailManager,rideHailAgent-047701-2013000697331-0-433937,BeamMobsim.iteration,RideHailManager,NotifyVehicleOutOfService(rideHailVehicle-0477...,NaN,-1,310374
2541753,message,RideHailManager,rideHailAgent-047701-2013000697331-0-433937,BeamMobsim.iteration,scheduler,"CompletionNotice(310374,Vector())",NaN,-1,310374


In [7]:
msg[
    (
            (msg['receiver_name'] == 'RideHailManager')
            | (msg['sender_name'] == 'RideHailManager')
    )
]


,type,sender_parent,sender_name,receiver_parent,receiver_name,payload,state,tick,triggerId
6179,message,BeamMobsim.iteration,RideHailManager,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
6180,message,BeamMobsim.iteration,RideHailManager,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
6181,message,BeamMobsim.iteration,RideHailManager,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
6182,message,BeamMobsim.iteration,RideHailManager,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
6183,message,BeamMobsim.iteration,RideHailManager,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
...,...,...,...,...,...,...,...,...,...
44171,event,BeamMobsim.iteration,RideHailManager,RideHailManager,rideHailAgent-2,Resume(11180),RideHailAgentData(rideHailVehicle-2@default (R...,-1,11180
44172,transition,BeamMobsim.iteration,RideHailManager,RideHailManager,rideHailAgent-2,IdleInterrupted,Idle,-1,-1076294169
44234,message,BeamMobsim.iteration,$b,BeamMobsim.iteration,RideHailManager,DebugRideHailManagerDuringExecution,NaN,-1,-1
44240,message,NaN,deadLetters,BeamMobsim.iteration,RideHailManager,DebugReport,NaN,-1,-1


In [48]:
# msg[msg['triggerId'] < 0]["triggerId"].unique().size
# msg[(msg['triggerId'] < 0) & (msg['type'] != 'transition')]
# msg[(msg['triggerId'] < 0) & (msg['payload'].str.startswith("RoutingResponse"))]
msg[(msg['triggerId'] == 2999)][ifields]
# msg[msg['payload'].str.startswith("RoutingRequest([x=553972.4563090717][y=4180774")]
# msg[msg['sender_name'].str.contains("rideHailAgent-228-L5")]
# msg.loc[90833, 'payload']

,type,sender_name,receiver_name,payload,state,tick,triggerId
7147,message,scheduler,ChargingNetworkManager,"TriggerWithId(ChargingTimeOutTrigger(1035,ride...",NaN,1035,2999
7168,event,ChargingNetworkManager,rideHailAgent-228-L5,"EndingRefuelSession(1035,rideHailVehicle-228-L...",RideHailAgentData(rideHailVehicle-228-L5@defau...,-1,2999
7169,message,rideHailAgent-228-L5,RideHailManager,"EndingRefuelSession(1035,rideHailVehicle-228-L...",NaN,-1,2999
7170,message,rideHailAgent-228-L5,ChargingNetworkManager,"ChargingUnplugRequest(1035,rideHailAgent-228-L...",NaN,-1,2999
7171,message,ChargingNetworkManager,ChargingNetworkManager,"ChargingPlugRequest(1035,rideHailVehicle-128-L...",NaN,-1,2999
7172,event,ChargingNetworkManager,rideHailAgent-228-L5,"UnpluggingVehicle(1035,rideHailAgent-228-L5,ri...",RideHailAgentData(rideHailVehicle-228-L5@defau...,1035,2999
7174,event,ChargingNetworkManager,rideHailAgent-128-L5,"StartingRefuelSession(1035,rideHailVehicle-128...",RideHailAgentData(rideHailVehicle-128-L5@defau...,-1,2999
7177,event,ChargingNetworkManager,rideHailAgent-128-L5,"StartingRefuelSession(1035,rideHailVehicle-128...",RideHailAgentData(rideHailVehicle-128-L5@defau...,-1,2999
7178,message,rideHailAgent-128-L5,RideHailManager,"StartingRefuelSession(1035,rideHailVehicle-128...",NaN,-1,2999
7179,message,rideHailAgent-128-L5,scheduler,"CompletionNotice(2999,Vector())",NaN,-1,2999


In [9]:
display("-- Special payload or other things --")

msg[
    # (msg['sender_parent'] == 'temp')
    # & (~msg['payload'].str.contains('RoutingRequest', na=False))
    # & (~msg['payload'].str.contains('MobilityStatusInquiry', na=False))
    (msg['payload'].str.contains('PlanEnergyDispatchTrigger', na=False))
    # & (msg['sender_name'] == '022802-2012001386215-0-6282252')
    # (msg['payload'].str.contains('RoutingRequest', na=False)) & (msg['triggerId'] == 8238)
    # (msg['receiver_parent'].str.contains('population', na=False))
    # (msg['receiver_parent'] == 'population')
    # (msg['sender_name'] == 'BeamMobsim.iteration' )
    ]

# msg

'-- Special payload or other things --'

,type,sender_parent,sender_name,receiver_parent,receiver_name,payload,state,tick,triggerId
12,message,BeamMobsim.iteration,ChargingNetworkManager,BeamMobsim.iteration,scheduler,"ScheduleTrigger(PlanEnergyDispatchTrigger(0),A...",NaN,-1,-1
13,message,BeamMobsim.iteration,scheduler,BeamMobsim.iteration,ChargingNetworkManager,"TriggerWithId(PlanEnergyDispatchTrigger(0),9)",NaN,0,9
143,message,BeamMobsim.iteration,ChargingNetworkManager,BeamMobsim.iteration,scheduler,"CompletionNotice(9,Vector(ScheduleTrigger(Plan...",NaN,-1,9
6230,message,BeamMobsim.iteration,scheduler,BeamMobsim.iteration,ChargingNetworkManager,"TriggerWithId(PlanEnergyDispatchTrigger(300),79)",NaN,300,79
6233,message,BeamMobsim.iteration,ChargingNetworkManager,BeamMobsim.iteration,scheduler,"CompletionNotice(79,Vector(ScheduleTrigger(Pla...",NaN,-1,79
...,...,...,...,...,...,...,...,...,...
43423,message,BeamMobsim.iteration,ChargingNetworkManager,BeamMobsim.iteration,scheduler,"CompletionNotice(11003,Vector(ScheduleTrigger(...",NaN,-1,11003
43761,message,BeamMobsim.iteration,scheduler,BeamMobsim.iteration,ChargingNetworkManager,TriggerWithId(PlanEnergyDispatchTrigger(58800)...,NaN,58800,11078
43763,message,BeamMobsim.iteration,ChargingNetworkManager,BeamMobsim.iteration,scheduler,"CompletionNotice(11078,Vector(ScheduleTrigger(...",NaN,-1,11078
43992,message,BeamMobsim.iteration,scheduler,BeamMobsim.iteration,ChargingNetworkManager,TriggerWithId(PlanEnergyDispatchTrigger(59100)...,NaN,59100,11147


In [ ]:
msg.loc[96167]['payload']

In [ ]:
# set sender, receiver types
import re

pattern = re.compile("^\d+(-\d+){1,}$")


def detectActor(parent, name):
    isParentPopulation = parent == "population"
    isParentHousehold = isinstance(parent, str) and parent.startswith("population/")
    isTransitDriverAgent = name.startswith("TransitDriverAgent-")
    isRideHailAgent = name.startswith("rideHailAgent-")
    looksLikeId = bool(pattern.match(name))
    if (isTransitDriverAgent): return "TransitDriverAgent"
    if (isRideHailAgent): return "RideHailAgent"
    if (isParentPopulation and looksLikeId): return "Household"
    if (isParentHousehold and looksLikeId): return "Person"
    if (isParentHousehold and ~looksLikeId): return f"HouseholdFleetManager:{name}"
    return name


filtered_msg = msg[(msg['sender_parent'] != 'temp')].copy()
filtered_msg['msg_type'] = filtered_msg['payload'].str.extract(r'TriggerWithId\(([^()]+)')
filtered_msg.loc[filtered_msg['msg_type'].isnull(), 'msg_type'] = filtered_msg['payload'].str.extract(r'(\w+)\(?')[0]
filtered_msg['sender'] = filtered_msg.apply(lambda row: detectActor(row['sender_parent'], row['sender_name']), axis=1)
filtered_msg['receiver'] = filtered_msg.apply(lambda row: detectActor(row['receiver_parent'], row['receiver_name']),
                                              axis=1)
filtered_msg.groupby(["sender", "receiver", "msg_type"]).count()

In [ ]:
filtered_msg[filtered_msg['triggerId'] < 0]['msg_type'].unique()
# filtered_msg[(filtered_msg['triggerId'] < 0) & (filtered_msg['msg_type'] == 'R5Network')]

In [ ]:
# filtered_msg[filtered_msg['sender'].str.startswith('BeamMobsim')]
filtered_msg[(filtered_msg['type'] != 'transition')]
    .groupby(["sender", "receiver", "msg_type"])['type'].count()
    .reset_index().rename(columns={'type': 'count'})
    .to_csv('beam_messages_count.csv')

In [ ]:
filtered_msg[(filtered_msg['type'] == 'transition')].groupby(['receiver', 'payload', 'state']).count()[['type', 'tick']]

In [ ]:
# creates triggerId -> person_id dictionary
def clean(s):
    return set([x for x in s if x >= 0])


rec = filtered_msg[(filtered_msg['receiver'] == "Person")].groupby('receiver_name')['triggerId'].agg(set)
rec.name = "rec"
sen = filtered_msg[(filtered_msg['sender'] == "Person")].groupby('sender_name')['triggerId'].agg(set)
sen.name = "sen"
rs = pd.concat([rec, sen], axis=1)
rs['trigger_ids'] = rs.apply(lambda row: clean(row['sen'].union(row['rec'])), axis=1)
# rs['trigger_ids']['010100-2012000596480-0-179584']
dict1 = rs['trigger_ids'].to_dict()
id_to_person = {id: person_id for person_id, trigger_ids in dict1.items() for id in trigger_ids}
# id_to_person = {k:v for (k, v) in id_person_pair}
# id_to_person
len(id_to_person)

In [ ]:
filtered_msg.iloc[146670:146690]

In [ ]:
#Clustering persons

def create_dimension(row):
    is_sender_person = row['sender'] == 'Person'
    is_receiver_person = row['receiver'] == 'Person'
    if is_sender_person and is_receiver_person: return f"{row['msg_type']}_self"
    if is_sender_person: return f"{row['msg_type']}_{row['receiver']}"
    if is_receiver_person: return f"{row['sender']}_{row['msg_type']}"
    raise ValueError("no person involved")



interactions = filtered_msg[filtered_msg['type'] != 'transition'].copy()
interactions.loc[interactions['sender'] == 'Person', 'person'] = interactions['sender_name']
interactions.loc[interactions['receiver'] == 'Person', 'person'] = interactions['receiver_name']
interactions = interactions[~interactions['person'].isnull()]
interactions['dimension'] = interactions.apply(create_dimension, axis=1)
interactions = interactions[['person', 'dimension']]
interactions

In [ ]:
persons = interactions.groupby(['person', 'dimension']).size().reset_index(name="num_msg").sort_values(['person', 'dimension'])
persons = persons.pivot(index='person', columns='dimension', values='num_msg')
persons.fillna(0, inplace=True)
persons

In [ ]:
persons_interaction_present = np.sign(persons)

In [ ]:
plt.figure(figsize=(8, 6), dpi=80)
dendrogram = sch.dendrogram(sch.linkage(persons_interaction_present, method  = "ward"))
plt.title('Dendrogram')
# plt.xlabel('Customers')
# plt.ylabel('Euclidean distances')
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering
hc = AgglomerativeClustering(n_clusters = 5, affinity = 'euclidean', linkage ='ward')

y_hc=hc.fit_predict(persons_interaction_present)


clusters = pd.DataFrame(y_hc, index=persons_interaction_present.index, columns=['cluster'])
shown = [
    '010600-2013000040063-0-1831035',
    '010200-2012000700304-0-6796854',
    '600900-2013000179752-0-8087176',
    '600300-2013000025013-0-200862',
    '010100-2012000596480-0-179582',
    '600800-2014001325153-3-578676',
         ]
display(clusters[clusters.index.isin(shown)])
display(clusters[clusters['cluster'] == 0])
clusters.groupby('cluster').size()